In [2]:
import requests
import zipfile
import io
import os 
import shutil
import pandas as pd
import logging
import configparser
import shutil
import random
import re
import yadisk


from pathlib import Path
from transliterate import translit
from datetime import datetime
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine, text, NullPool

import warnings
warnings.filterwarnings("ignore")

In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')
ssh_host = config['database']['ssh_host_test']
ssh_port = int(config['database']['ssh_port'])
ssh_username = config['database']['ssh_username']
ssh_password = config['database']['ssh_password']
database_username = config['database']['database_username']
database_password = config['database']['database_password']
database_name = config['database']['database_name']
localhost = config['database']['localhost']
localhost_port = int(config['database']['localhost_port'])
table_name = config['database']['table_name']
ya_api = config['yandex']['ya_api']
ya_link = config['yandex']['ya_link']
ya_token = config['yandex']['ya_token']

# create savedir

In [5]:
def create_save_dir_check():
    try:
        Path.mkdir(Path.cwd()/'check_csv')
    except:
        pass
    save_dir = (Path.cwd()/'check_csv').as_posix()
    return save_dir

In [6]:
local_save_dir = create_save_dir_check()

In [7]:
local_save_dir

'C:/Users/pnknv/Documents/GitHub/yadisk_loader/check_csv'

# load files

## load personal disk

In [8]:
y = yadisk.YaDisk(token=ya_token)

In [9]:
if y.check_token():
    print('token valid')
else:
    print('token invalid')
    print('перейдите по ссылке: https://oauth.yandex.ru/authorize?response_type=token&client_id=<> вставив свой айди приложения')
    print('id приложения можно получить в: https://oauth.yandex.ru/ - пользователь yanedvizhkatop')
    print('мануал: https://ramziv.com/article/8')

token valid


In [10]:
disk_files_info = list(y.listdir("/cian"))

In [11]:
disk_files = [disk_files_info[i].path for i in range(len(disk_files_info))]

In [12]:
disk_files

['disk:/cian/17-03-23.csv',
 'disk:/cian/18-03-23.csv',
 'disk:/cian/20-03-23.csv',
 'disk:/cian/21-03-23.csv']

In [13]:
Path(local_save_dir)/Path(disk_files[-1]).name

WindowsPath('C:/Users/pnknv/Documents/GitHub/yadisk_loader/check_csv/21-03-23.csv')

In [14]:
y.download(disk_files[-1], (Path(local_save_dir)/Path(disk_files[0]).name).as_posix())

<ResourceLinkObject{'href': 'https://cloud-api.yandex.net/v1/disk/resources?path=disk%3A%2Fcian%2F21-03-23.csv', 'method': 'GET', 'templated': False, 'path': 'disk:/cian/21-03-23.csv'}>

In [15]:
Path(disk_files[-1]).name

'21-03-23.csv'

In [39]:
local_save_dir+'/'+Path(disk_files[0]).name

'C:/Users/pnknv/Documents/GitHub/yadisk_loader/check_csv/17-03-23.csv'

In [18]:
cian_dima = pd.read_csv(local_save_dir+'/'+Path(disk_files[0]).name, encoding='cp1251', delimiter=',')

In [19]:
cian_dima.drop_duplicates(subset=['link'], keep='last', inplace=True)
cian_dima.reset_index(drop=True, inplace=True)

In [20]:
cian_dima

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15


## load ilya disk

In [21]:
# скачал файлы из яндекс диска https://disk.yandex.ru/d/sVgz0GiqjhCsNg без алгоритма

In [23]:
local_save_dir

'C:/Users/pnknv/Documents/GitHub/yadisk_loader/check_csv'

In [24]:
cian_ilya = pd.read_csv(local_save_dir+'/циан 20-03-23 (без дублей).csv', delimiter=';', encoding='utf8')

In [25]:
cian_ilya.drop_duplicates(subset=['Ссылка'], keep='last', inplace=True)
cian_ilya.reset_index(drop=True, inplace=True)

In [26]:
cian_ilya['city'] = cian_ilya['Адрес'].apply(lambda x: x.split(', ')[1])

# analyze

In [120]:
list_cities = ['Краснодар', 'Казань', 'Анапа', 'Челябинск', 'Новороссийск', 'Томск', 'Армавир']

In [131]:
(65644 - 64030) / 65644

0.02458716714398879

In [121]:
cian_ilya.city.value_counts().head(20)

Краснодар                                 30651
Казань                                    10853
Анапа                                      8447
Челябинск                                  4932
Новороссийск                               4786
Томск                                      4184
Армавир                                     177
Новороссийск муниципальное образование        6
Анапа муниципальное образование               2
Динской район                                 1
Сосновский район                              1
Сочи                                          1
Краснодар муниципальное образование           1
Name: city, dtype: int64

In [122]:
print('всего уникальных записей у Ильи:',len(cian_ilya))

всего уникальных записей у Ильи: 64042


In [123]:
print('Из них в списке целевых городов:', len(cian_ilya[cian_ilya.city.isin(list_cities)]))

Из них в списке целевых городов: 64030


In [124]:
cian_dima.city.value_counts()

Краснодар                            31194
Сочи                                 23462
Казань                               11243
Анапа                                 8263
Новороссийск                          5511
                                     ...  
Краснодарский                            1
Железнодорожного разъезда Киндери        1
Березовка                                1
Полевой                                  1
Барабинка                                1
Name: city, Length: 134, dtype: int64

In [125]:
cian_dima[cian_dima.city.isin(list_cities)].city.value_counts()

Краснодар       31194
Казань          11243
Анапа            8263
Новороссийск     5511
Челябинск        5018
Томск            4308
Армавир           107
Name: city, dtype: int64

In [126]:
print('всего уникальных записей у Димы:',len(cian_dima))

всего уникальных записей у Димы: 100830


In [127]:
print('Из них в списке целевых городов:', len(cian_dima[cian_dima.city.isin(list_cities)]))

Из них в списке целевых городов: 65644


In [144]:
list_cities_dima = ['Краснодар', 'Казань', 'Анапа', 'Челябинск', 'Новороссийск', 'Томск', 'Армавир', 'Сочи']

In [145]:
cian_dima[cian_dima.city.isin(list_cities_dima)].city.value_counts()

Краснодар       31194
Сочи            23462
Казань          11243
Анапа            8263
Новороссийск     5511
Челябинск        5018
Томск            4308
Армавир           107
Name: city, dtype: int64

In [126]:
print('всего уникальных записей у Димы:',len(cian_dima))

всего уникальных записей у Димы: 100830


In [146]:
print('Из них в списке целевых городов + Сочи:', len(cian_dima[cian_dima.city.isin(list_cities_dima)]))

Из них в списке целевых городов + Сочи: 89106


# районы

In [35]:
cian_dima.iloc[10000,:]

source                                                       cian
ad_id                                                   280613941
addr            Челябинская область; Челябинск; р-н Трактороза...
city                                                    Челябинск
raion                                           Тракторозаводский
mikroraion                                               Чурилово
square                                                       58.9
floor                                                           6
house_floors                                                    9
type                                                       2 комн
link             https://chelyabinsk.cian.ru/sale/flat/280613941/
price                                                     3650000
price_sqrm                                                  61969
offer_from                                            Собственник
date                                          2022-11-27 20:04:06
Name: 1000

In [34]:
cian_dima.iloc[10000,2]

'Челябинская область; Челябинск; р-н Тракторозаводский; мкр. Чурилово; улица Трашутина; 29'

In [28]:
cian_dima['raion'] = cian_dima.addr.apply(lambda x: x.split(';'))

,source,ad_id,addr,city,raion,mikroraion,square,floor,house_floors,type,link,price,price_sqrm,offer_from,date
0,cian,210197059,Краснодарский край; Анапа; улица Горького; 2А,Анапа,NaN,NaN,107.0,7,12,3 комн,https://anapa.cian.ru/sale/flat/210197059/,25000000,233645,Собственник,2019-06-16 19:18:17
1,cian,213575526,Краснодарский край; Анапа; улица Краснозеленых...,Анапа,NaN,NaN,106.0,6,7,3 комн,https://anapa.cian.ru/sale/flat/213575526/,10100000,95283,Риелтор,2019-07-26 11:52:55
2,cian,232012292,Краснодарский край; Анапа; улица Ленина; 194,Анапа,NaN,NaN,20.0,1,9,Студия,https://anapa.cian.ru/sale/flat/232012292/,2100000,105000,Риелтор,2020-05-14 15:01:50
3,cian,233965925,Краснодарский край; Анапа; улица Толстого; 81,Анапа,NaN,NaN,29.0,1,3,1 комн,https://anapa.cian.ru/sale/flat/233965925/,4500000,155172,Агентство недвижимости,2020-06-08 14:04:52
4,cian,239179225,Краснодарский край; Анапа; Владимирская улица;...,Анапа,NaN,NaN,140.0,13,13,5 комн,https://anapa.cian.ru/sale/flat/239179225/,15300000,109286,Собственник,2020-08-17 16:40:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101775,cian,285075402,Томская область; Томск; р-н Советский; проспек...,Томск,Советский,NaN,18.0,5,5,1 комн,https://tomsk.cian.ru/sale/flat/285075402/,1950000,108333,Агентство недвижимости,2023-03-21 12:35:00
101776,cian,285075572,Томская область; Томск; р-н Советский; улица Ш...,Томск,Советский,NaN,16.5,2,5,1 комн,https://tomsk.cian.ru/sale/flat/285075572/,2000000,121212,Агентство недвижимости,2023-03-21 12:39:13
101777,cian,285076163,Томская область; Томск; р-н Ленинский; улица Р...,Томск,Ленинский,NaN,48.0,5,6,2 комн,https://tomsk.cian.ru/sale/flat/285076163/,4350000,90625,Агентство недвижимости,2023-03-21 12:48:54
101778,cian,285076291,Томская область; Томск; р-н Ленинский; Черемош...,Томск,Ленинский,NaN,39.0,4,10,1 комн,https://tomsk.cian.ru/sale/flat/285076291/,3680000,94359,Агентство недвижимости,2023-03-21 12:51:15
